+ boosting是一族可将弱学习器升级为强学习器的算法。先从初始数据训练一个基学习器，再根据基学习器的表现对训练样本分布进行调整，使得先前基学习器做错的样本在后续获得更多的关注，然后基于调整后的样本分布来训练下一个基学习器，如此往复，直至基学习器的数目达到指定的数目T，再将这T个基学习器进行加权结合。
+ bagging是通过放回随机采样的方式，采样出T个含m个样本的采样集，然后针对采样集训练一个基学习器，再将这些学习器进行结合。通常对分类任务采用简单投票法，对回归任务采用简单平均法。
+ stacking（树行）是指通过初级学习器和次级学习器组合进行预测的一种方式。与bagging（并行），boosting（串行）并列。
具体原理我们先将数据分为训练集M和测试集N，对训练集进行k折交叉验证，然后对测试集进行预测。得到交叉验证中的k个验证集结果，将k个验证集的预测结果合并得到M个预测结果M1，。同时对测试集预测，得到k个测试结果，对此平均处理N1。
综上，得到了一个模型的结果。
然后在使用其他模型得出（M2，N2），（M3，N3）……
然后使用[M1，M2，M3……，Y]作为训练集，[N1，N2，N3……，Y]作为训练集进行预测
![stacking](./stacking.png)

In [4]:
# coding: utf-8
# pylint: disable = invalid-name, C0111
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('./regression.train', header=None, sep='\t')
df_test = pd.read_csv('./regression.test', header=None, sep='\t')

y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Loading data...
Starting training...
[1]	valid_0's l1: 0.491812	valid_0's l2: 0.24288
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.48798	valid_0's l2: 0.239307
[3]	valid_0's l1: 0.483905	valid_0's l2: 0.235559
[4]	valid_0's l1: 0.478533	valid_0's l2: 0.230771
[5]	valid_0's l1: 0.47333	valid_0's l2: 0.226297
[6]	valid_0's l1: 0.470079	valid_0's l2: 0.223692
[7]	valid_0's l1: 0.466768	valid_0's l2: 0.220941
[8]	valid_0's l1: 0.462971	valid_0's l2: 0.217982
[9]	valid_0's l1: 0.459474	valid_0's l2: 0.215351
[10]	valid_0's l1: 0.45619	valid_0's l2: 0.213064
[11]	valid_0's l1: 0.4532	valid_0's l2: 0.211053
[12]	valid_0's l1: 0.450526	valid_0's l2: 0.209336
[13]	valid_0's l1: 0.447621	valid_0's l2: 0.207492
[14]	valid_0's l1: 0.445178	valid_0's l2: 0.206016
[15]	valid_0's l1: 0.442718	valid_0's l2: 0.204677
[16]	valid_0's l1: 0.440293	valid_0's l2: 0.203224
[17]	valid_0's l1: 0.437015	valid_0's l2: 0.201186
[18]	valid_0's l1: 0.434725	valid_0's l2: 0.199626

Starting predicting...
The rmse of prediction is: 0.44427172467679765
